In [1]:
import requests
import pandas as pd
import numpy as np
import time
from bs4 import BeautifulSoup
import random
import os
import datetime


In [2]:
stock ={"陽明":"2609","長榮":"2603","萬海":"2615","長榮航":"2618","遠雄港":"5607","華航":"2610","台驊投控":"2636","裕民":"2606",
        "益航":"2601" ,"東森":"2614","新興":"2605" , "四維航":"5608" , "台航":"2617" }
month_now =datetime.datetime.strftime(datetime.datetime.today() ,'%Y%m')+"01" #現在的月份，url所使用
month_index = datetime.datetime.strftime(datetime.datetime.today() ,'%Y%m') #index使用
url_price = [f"https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=html&date={month_now}&stockNo={code}" for code in stock.values() ]


### 取每月的最高價、最低價、收盤中位數、成交量中位數

In [3]:
#header
headers ={"user-agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"}


In [4]:
def get_request_price(url   ,headers ):
    """負責request的請求
    傳入網址、header(避免反爬蟲機制)"""
    r = requests.get(url, headers=headers )
    r.encoding="utf-8"
    res = r.text
    return res

In [5]:
#股價
def get_value_price(reponse):
    """請傳入 rqquest.get(url).text"""
    
    """return 資料為一個月的數值的四個數值 max_price、min_price、median_price、median的交易筆數 """
    #使用BeautifulSoup 做解析
    from bs4 import BeautifulSoup
    soup=BeautifulSoup( reponse ,"html5lib")
    max_p = []#每日最大的價格的集合（月）
    min_p=[]#每日最低的價格的集合（月）
    median_p=[]#每日的收盤價格的集合（月）
    median_trans_all =[]#每日的交易筆數的集合（月）
      
    for  tr in soup.find_all("tr")[2:]:
        if tr.find_all("td")[4].text == "--": #當其中值是等於該“"--"符號，跳過本次迴圈
            continue
    
        max_=float(tr.find_all("td")[4].text)#原欄位是當天最高價
        min_=float(tr.find_all("td")[5].text)#原欄位是當天最低價
        median_=float(tr.find_all("td")[6].text)#原欄位是當天收盤價
        median_tran=float(tr.find_all("td")[8].text.replace(",",""))#去除逗號、原欄位是當天的交易筆數
        max_p.append(max_)
        min_p.append(min_)
        median_p.append(median_)
        median_trans_all.append(median_tran)

    max_end =np.max(max_p)#將該當月最高價的list，找出當月的最高價
    min_end =np.min(min_p)#將該當月最低價的list，找出當月的最低價
    median_price_end=np.median(median_p)#將該當月的收盤價list，找出當月的中位數收盤價
    median_tran_end =np.median(median_trans_all)#將該當月最低價的交易筆數list，找出當月的中位數交易筆數
    
    return  max_end ,  min_end ,  median_price_end ,  median_tran_end

In [6]:
df_plus=[]

for url_p  in url_price :
    
    print(url_p)
    res_price = get_request_price(url_p ,headers )
    time.sleep(3)

    max_end ,  min_end ,  median_price_end ,  median_tran_end=get_value_price(res_price)

    df=pd.DataFrame({"max_price":max_end,"min_price":min_end,"medain_price":median_price_end,
                      "medain_transactions":median_tran_end },index= [month_index]).round(2)
    df_plus.append(df)

https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=html&date=20210601&stockNo=2609
https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=html&date=20210601&stockNo=2603
https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=html&date=20210601&stockNo=2615
https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=html&date=20210601&stockNo=2618
https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=html&date=20210601&stockNo=5607
https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=html&date=20210601&stockNo=2610
https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=html&date=20210601&stockNo=2636
https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=html&date=20210601&stockNo=2606
https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=html&date=20210601&stockNo=2601
https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=html&date=20210601&stockNo=2614
https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=html&date=20210601&stockNo=2605
https://ww

In [7]:
#將新的資訊加入原有的資料中
for new ,  key ,name in zip(df_plus  , stock.keys() ,stock.values()) :
    #1.新比對月份是否有重複的
    old=pd.read_csv(f"./Data_price/{key}_{name}.csv",index_col=["Unnamed: 0"])#讀入舊的dataFRAME
    new_index=int(new.index.values)
    old_index=old.tail(1).index.values
    if new_index == old_index: 
        """檢查最後ㄧ列的月份是否相同，如相同drop掉，用新的資料"""
        old.drop(index =old_index ,inplace=True )
        #2.將新的月份資料結合
        new_= pd.concat([old ,new])
        
    else:
         #2.將新的月份資料結合
        new_= pd.concat([old ,new])
    
    new_.to_csv(f"./Data_price/{key}_{name}.csv")
        
        
    